### **GloVe**  
임베딩 벡터의 내적이 말뭉치 전체에서의 동시출현(co-occurrence) 확률 값이 되는 목적 함수를 갖는다.  
이를 통해 임베딩 벡터간 유사도 측정을 수월하게 하면서도 말뭉치 전체의 통계 정보를 반영할 수 있다. 

> **동시 출현 (Co-occurence)** *이란, 한 문장, 문단 또는 텍스트 단위에서 같이 출현한 단어를 가리다. 언어학적 의미에서 의미적 근접성을 가리킨다.*  




In [1]:
!pip install glove_python

     |████████████████████████████████| 266kB 5.0MB/s 
  Created wheel for glove-python: filename=glove_python-0.1.0-cp36-cp36m-linux_x86_64.whl size=700271 sha256=38af035eb4b1990f512b5b024ea5c700069a772fe97fbe49022a2fdfc4a4be7e
  Stored in directory: /root/.cache/pip/wheels/88/4b/6d/10c0d2ad32c9d9d68beec9694a6f0b6e83ab1662a90a089a4b
Successfully built glove-python


In [2]:
import os
import re

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
BASE_DIR = "/content/gdrive/My Drive/Colab Notebooks/ETRI_Article_Summarizer/Text-preprocessing-Data/articles"
ORIGIN_PATH = os.path.join(BASE_DIR,"Origin-Data")
PREPROCESSED_PATH = os.path.join(BASE_DIR,"Preprocessed-Data")
PRETTY_PATH = os.path.join(BASE_DIR,"Pretty-Data")
SWORDS_PATH = os.path.join(BASE_DIR, "StopWordList.txt")

In [5]:
class RawTextReader:
    def __init__(self, filepath):
        self.filepath = filepath
        self.rgxSplitter = re.compile("/n")

    def __iter__(self):
        for line in open(self.filepath, encoding='utf-8'):
            ch = self.rgxSplitter.split(line)
            for s in ch:
                yield s

기사 본문 내용이 짧기 때문에, 모델을 학습하기에 corpus의 크기가 작다.  
아래는 수집한 기사 87건들을 통해 corpus를 구성하고, GloVe 모델을 구축하는 내용이다.

In [6]:
media_list = os.listdir(ORIGIN_PATH)

result = []
forCount = []
for media in media_list:
    media_path = os.path.join(PREPROCESSED_PATH, media)
    article_list= os.listdir(media_path)

    for article in article_list:
        reader = RawTextReader(os.path.join(media_path, article)) 
        content = list(filter(None, reader))
        forCount += [token for sent in content for token in sent.split()]
        result += [sent.split() for sent in content]

In [7]:
print("전체 token의 개수 : {len}".format(len=len(forCount)))
print("중복되지 않은 token의 개수 : {len}".format(len=len(list(set(forCount)))))

전체 token의 개수 : 15668
중복되지 않은 token의 개수 : 2877


In [8]:
from glove import Corpus, Glove

corpus = Corpus() 
corpus.fit(result, window=5)

In [10]:
glove = Glove(no_components=100, learning_rate=0.05)
glove.fit(corpus.matrix, epochs=20, no_threads=4, verbose=True)
glove.add_dictionary(corpus.dictionary)

Performing 20 training epochs with 4 threads
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19


경찰서장과 비슷한 의미의 단어를 출력한다.

In [13]:
model_result1=glove.most_similar("경찰서장")
print(model_result1)

[('서장', 0.866163212925113), ('보고', 0.8591953308235412), ('병력', 0.8552500263463577), ('8월', 0.8545198920836846)]
